In [ ]:
import pandas as pd
import numpy as np

# 1. 아이템기반 추천시스템 
* 프로젝트 초반에만 적용됨 - 그 이후는 유저기반이나 다른걸 사용한다.

In [ ]:
pd.set_option("max_colwidth", 100)      # 칸이 보이지 않아서 넓이를 지정해주는 함수

# 데이터를 불러온다.
df = pd.read_csv(
    "./data/tmdb_5000_movies.csv",
)
df_orig = df.copy()

df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret...",en,Spectre,A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""name"": ""Danjaq"", ""id"": 10761}, {""name"": ""B24"", ""id"": ...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}, {""iso_3166_1"": ""US"", ""name"": ""United States of ...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}, {""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": ...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""te...",en,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c...",112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""name"": ""Warner Bros."", ""id"": 6194}, {""name"": ""DC E...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fic...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": 1456, ""name"": ""medal...",en,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the myster...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of Ameri

In [ ]:
df.info()               # 데이터에 대한 대략적인 정보 보기

<class 'pandas.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   str    
 2   homepage              1712 non-null   str    
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   str    
 5   original_language     4803 non-null   str    
 6   original_title        4803 non-null   str    
 7   overview              4800 non-null   str    
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   str    
 10  production_countries  4803 non-null   str    
 11  release_date          4802 non-null   str    
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   str    
 15  status                4803 non-n

In [ ]:
df.isna().sum()         # 전체 결측치 확인

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [ ]:
df.iloc[0]["genres"]        # 어떤 데이터가 들어있는지 확인하기 위해 찍어봄. str데이터인걸 확인

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [ ]:
from ast import literal_eval

result = literal_eval(df.iloc[0]["genres"])     # str인 다른 데이터를 바로 사용할 수 있는 원래 데이터로 바꿔줌.
result

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [ ]:
df["genres"] = df_orig["genres"].apply(literal_eval)  # 이렇게 해야 pandas 내부 함수이기 때문에 for문보다 훨씬 빠름

df.info()

<class 'pandas.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   str    
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   str    
 5   original_language     4803 non-null   str    
 6   original_title        4803 non-null   str    
 7   overview              4800 non-null   str    
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   str    
 10  production_countries  4803 non-null   str    
 11  release_date          4802 non-null   str    
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   str    
 15  status                4803 non-n

In [ ]:
# 구조 파악하기
data_list = []

for i in df.iloc[0,:]["genres"]:
    data_list.append(i['name'])

data_list       # 이렇게 생겼구나 확인

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [ ]:
# 장르 리스트화
df["genres"] = df["genres"].apply(          # list comprehension으로 딕셔너리에 있는 값들을 꺼내서 리스트화
    lambda lst: [x["name"] for x in lst]
)

df["genres"].head()

0    [Action, Adventure, Fantasy, Science Fiction]
1                     [Adventure, Fantasy, Action]
2                       [Action, Adventure, Crime]
3                 [Action, Crime, Drama, Thriller]
4             [Action, Adventure, Science Fiction]
Name: genres, dtype: object

In [69]:
# SF와 같은 띄어쓰기로 되어있는 데이터 붙이기.
df["genres"] = df["genres"].apply(
    lambda genre_list: [''.join(g.split()) for g in genre_list]
)

df["genres"].head()

0    [Action, Adventure, Fantasy, ScienceFiction]
1                    [Adventure, Fantasy, Action]
2                      [Action, Adventure, Crime]
3                [Action, Crime, Drama, Thriller]
4             [Action, Adventure, ScienceFiction]
Name: genres, dtype: object

In [70]:
# CountVectorizer에 넣을 수 있게 prep하기.
df["genres"] = df["genres"].apply(
    lambda genre_list: ' '.join(genre_list)
)

df["genres"].head()

0    Action Adventure Fantasy ScienceFiction
1                   Adventure Fantasy Action
2                     Action Adventure Crime
3                Action Crime Drama Thriller
4            Action Adventure ScienceFiction
Name: genres, dtype: str

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer 모델 준비 - 단어사전을 만들고 등장한 단어 개수를 counting해준다.
count_vec = CountVectorizer(
    min_df=1, ngram_range=(1,1)
)

# 학습하여 변환 적용
genre_mat = count_vec.fit_transform(df["genres"])

In [ ]:
# 벡터라이저의 단어사전 확인하기 - 인덱스의 순서는 단어의 alphabetical order. 알파벳순
count_vec.vocabulary_

{'action': 0,
 'adventure': 1,
 'fantasy': 8,
 'sciencefiction': 15,
 'crime': 4,
 'drama': 6,
 'thriller': 16,
 'animation': 2,
 'family': 7,
 'western': 19,
 'comedy': 3,
 'romance': 14,
 'horror': 11,
 'mystery': 13,
 'history': 10,
 'war': 18,
 'music': 12,
 'documentary': 5,
 'foreign': 9,
 'tvmovie': 17}

In [ ]:
# 넘파이 배열로 표현하기 - 등장한 단어 개수 카운팅
genre_mat.toarray()

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4803, 20))

In [ ]:
# 배열을 데이터프레임으로 만들기, 
df_genre_vec = pd.DataFrame(genre_mat.toarray())
df_genre_vec.columns = sorted(count_vec.vocabulary_, key=count_vec.vocabulary_.get) # 단어의 순서대로 열에 적용한다.
df_genre_vec.index = df["title"]
df_genre_vec.head()

,action,adventure,animation,comedy,crime,documentary,drama,family,fantasy,foreign,history,horror,music,mystery,romance,sciencefiction,thriller,tvmovie,war,western
title,,,,,,,,,,,,,,,,,,,,
Avatar,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
Pirates of the Caribbean: At World's End,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
Spectre,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
The Dark Knight Rises,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
John Carter,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(df_genre_vec, df_genre_vec)   # 코사인 유사도로 비교한다.
genre_sim

array([[1.        , 0.8660254 , 0.57735027, ..., 0.        , 0.        ,
        0.        ],
       [0.8660254 , 1.        , 0.66666667, ..., 0.        , 0.        ,
        0.        ],
       [0.57735027, 0.66666667, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]], shape=(4803, 4803))

In [ ]:
genre_sim_df = pd.DataFrame(genre_sim)      # 유사도 표를 데이터프레임으로 만들어서 볼 수 있게 한다.
genre_sim_df.columns = df["title"]          # 영화 제목이 행과 열이기 때문
genre_sim_df.index = df["title"]
genre_sim_df

title,Avatar,Pirates of the Caribbean: At World's End,Spectre,The Dark Knight Rises,John Carter,Spider-Man 3,Tangled,Avengers: Age of Ultron,Harry Potter and the Half-Blood Prince,Batman v Superman: Dawn of Justice,...,On The Downlow,Sanctuary: Quite a Conundrum,Bang,Primer,Cavite,El Mariachi,Newlyweds,"Signed, Sealed, Delivered",Shanghai Calling,My Date with Drew
title,,,,,,,,,,,,,,,,,,,,,
Avatar,1.000000,0.866025,0.577350,0.250000,0.866025,0.866025,0.0,0.866025,0.577350,0.866025,...,0.0,0.000000,0.0,0.288675,0.000000,0.288675,0.000000,0.000000,0.0,0.0
Pirates of the Caribbean: At World's End,0.866025,1.000000,0.666667,0.288675,0.666667,1.000000,0.0,0.666667,0.666667,1.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.333333,0.000000,0.000000,0.0,0.0
Spectre,0.577350,0.666667,1.000000,0.577350,0.666667,0.666667,0.0,0.666667,0.333333,0.666667,...,0.0,0.000000,0.0,0.000000,0.000000,0.666667,0.000000,0.000000,0.0,0.0
The Dark Knight Rises,0.250000,0.288675,0.577350,1.000000,0.288675,0.288675,0.0,0.288675,0.000000,0.288675,...,0.5,0.288675,0.5,0.577350,0.353553,0.866025,0.000000,0.250000,0.0,0.0
John Carter,0.866025,0.666667,0.666667,0.288675,1.000000,0.666667,0.0,1.000000,0.333333,0.666667,...,0.0,0.000000,0.0,0.333333,0.000000,0.333333,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
El Mariachi,0.288675,0.333333,0.666667,0.866025,0.333333,0.333333,0.0,0.333333,0.000000,0.333333,...,0.0,0.333333,0.0,0.333333,0.408248,1.000000,0.000000,0.000000,0.0,0.0
Newlyweds,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.408248,0.0,0.000000,0.000000,0.000000,1.000000,0.707107,0.0,0.0
"Signed, Sealed, Delivered",0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.5,0.288675,0.5,0.288675,0.000000,0.000000,0.707107,1.000000,0.0,0.0


In [ ]:
genre_sim_df.iloc[0].sort_values(ascending=False) # 0번째 행에 있는 영화제목과 유사도값들을 내림차순으로 불러옴

title
Avatar                         1.0
Superman Returns               1.0
Man of Steel                   1.0
Jupiter Ascending              1.0
Superman II                    1.0
                              ... 
Daddy's Home                   0.0
Love in the Time of Cholera    0.0
Stepmom                        0.0
The Game                       0.0
Dry Spell                      0.0
Name: Avatar, Length: 4803, dtype: float64

In [ ]:
query = "Superman II"

index_no = df.loc[df["title"]==query,:].index[0]    # 쿼리를 만들어서 이런식으로 함수화가 가능함.

genre_sim_df.iloc[index_no].sort_values(ascending=False)

title
Avatar                         1.0
Superman Returns               1.0
Man of Steel                   1.0
Jupiter Ascending              1.0
Superman II                    1.0
                              ... 
Daddy's Home                   0.0
Love in the Time of Cholera    0.0
Stepmom                        0.0
The Game                       0.0
Dry Spell                      0.0
Name: Superman II, Length: 4803, dtype: float64